In [1]:
import requests
import os 
import io
import pandas as pd 
from PIL import Image
from bs4 import BeautifulSoup

In [ ]:
base_url = 'url-{}.htm'
# Paste the url you want in the "url" section
firma_bilgileri = {}
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36"}
url=''

In [ ]:
href_listesi = []
for page_number in range(1,25):  
    if page_number == 1:
        url = 'url'
        # Paste the url you want in the "url" section
    else:
        url = base_url.format(page_number)      
    sayfa = requests.get(url, headers=headers)
    soup = BeautifulSoup(sayfa.text, "html.parser")
    List = soup.find("div", {"class": "clearfix"}).find_all("a", {"class": "corp-name"})
    for a in soup.find_all('a', class_='corp-name'):
        href = a.get('href')
        href_listesi.append(href)
    for href in href_listesi:
        print(href)

In [ ]:
base_url_2='url/{}'

In [ ]:
firma_url_liste = []
for firma in href_listesi:  
    url_firma=base_url_2.format(firma)
    firma_url_liste.append(url_firma)

In [ ]:
for firma in firma_url_liste:
    print(firma)

In [ ]:
sayfa2 = requests.get(url_firma,headers=headers)
soup2=BeautifulSoup(sayfa2.text , "html.parser")

In [ ]:
firma_bilgileri_listesi = []
for url in firma_url_liste:
    firma_bilgileri = {}  # Her bir URL için firma bilgilerini tutacak sözlüğü yeniden başlatın
    
    # URL'yi kullanarak gerekli işlemleri gerçekleştirin ve firma bilgilerini alın
    # Örneğin, aşağıdaki kodlar HTML sayfasından firma bilgilerini alıyor gibi görünüyor:
    sayfa2 = requests.get(url, headers=headers)
    soup2 = BeautifulSoup(sayfa2.text, "html.parser")

    for tr in soup2.find_all('tr'):
        tds = tr.find_all('td')
        if len(tds) == 3:
            key = tds[0].text.strip()
            value = tds[2].text.strip()
            firma_bilgileri[key] = value
    firma_bilgileri_listesi.append(firma_bilgileri)
    # Her bir firma için bilgileri yazdırın
    for key, value in firma_bilgileri.items():
        print(f"{key}: {value}")

    print("\n")  # Her bir firma bilgisi seti arasına boş bir satır ekleyin

In [ ]:
df = pd.DataFrame(firma_bilgileri_listesi)
df

In [ ]:
image_urls = []

for page_number in range(1, 26):  
    if page_number == 1:
        url = 'url/firmalar.htm'
    else:
        url = base_url.format(page_number)
        
    response = requests.get(url)
    html_content = response.text
    soup = BeautifulSoup(html_content, "html.parser")

    for a_tag in soup.find_all('a', class_='corp-image'):
        img_tag = a_tag.find('img')
        if img_tag:       
            img_url = img_tag['src']   
            image_urls.append(img_url)

In [ ]:
for img_url in image_urls:
    print(img_url)
len(image_urls)

In [ ]:
df['images'] = image_urls
df

In [ ]:
firma_aciklama_listesi = []

for url in firma_url_liste:
    aciklama = ''      
    sayfa3 = requests.get(url, headers=headers)
    soup3 = BeautifulSoup(sayfa3.text, "html.parser")
    
    # Açıklamaların bulunduğu HTML yapısını inceleyerek gerekli etiketleri bulalım
    aciklama_etiketleri = soup3.find_all('div', id='detail-wrap')
   
    # Her bir açıklama etiketini dolaşarak içeriğini birleştirelim
    for etiket in aciklama_etiketleri:
        paragraflar = etiket.find_all('p')  # Tüm <p> etiketlerini al
        for p in paragraflar:
            aciklama += p.get_text(strip=True) + '\n'  # Her bir paragrafın metnini alarak birleştir
    
    # Elde edilen açıklamayı listeye ekleyelim
    firma_aciklama_listesi.append(aciklama)
    
# Her bir URL için alınan açıklamaları ekrana yazdıralım
for aciklamaa in firma_aciklama_listesi:
    print(aciklamaa)

In [ ]:
df['aciklama'] = firma_aciklama_listesi
df

In [ ]:
df['Gsm No'] = df['Gsm No'].astype(str)

df['Gsm No'] = df['Gsm No'].str.replace(' ', '')

df['Faks'] = df['Faks'].astype(str)

df['Faks'] = df['Faks'].str.replace(' ', '')

df['Telefon'] = df['Telefon'].astype(str)

df['Telefon'] = df['Telefon'].str.replace(' ', '')

df

In [ ]:
df[['Şehir', 'İlçe']] = df['Şehir / İlçe'].str.split(' - ', expand=True)
df.drop(columns=['Şehir / İlçe'], inplace=True)

In [ ]:
df['Şehir'] =df['Şehir'].astype(str)
df['İlçe'] =df['İlçe'].astype(str)
df['Şehir'] = df['Şehir'].str.strip()
df['İlçe'] = df['İlçe'].str.strip()

df.dtypes
df

In [ ]:
df = df.fillna("nan")
df

In [ ]:
df['Telefon'] = df['Telefon'].astype(str)
df['Faks'] = df['Faks'].astype(str)
df['Gsm No'] = df['Gsm No'].astype(str)
df.to_parquet('firma.parquet', index=False)

In [ ]:
folder_path = 'images_firma'
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
base_url_img = 'url/'
full_urls = [base_url_img + url for url in image_urls]

In [ ]:
for index,imgs in enumerate(full_urls, start=1):
    urll = imgs
    if "100x60-logo.jpg" not in urll:
        try:
        
            # URL'den fotoğrafı indir
            response = requests.get(urll)                
            print(response)
            print(urll)
            # Fotoğrafı dosyaya kaydet
            with open(f'{folder_path}/{index}-image.jpg', 'wb') as f:
                f.write(response.content)
        
            print(f'Image {index} downloaded successfully!')
        except Exception as e:
            print(f'Error downloading image {index}: {e}')